In [1]:
!pip install torch transformers protobuf sentencepiece


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")



Using device: mps


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Make sure sentencepiece is installed and use the slow tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

question = "John is twice as old as Mary. Mary is 10 years old. How old is John?"
prompt = f"Q: {question}\nA: Let's think step by step.\n"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Chain of Thought Reasoning ===")
print(result)


/Users/vaibhav/PycharmProjects/workfile/.venv1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== Chain of Thought Reasoning ===
Mary is 10 - 10 = 10 years old. John is 10 - 10 = 10 years old. The answer: 10.


In [2]:
!pip install bertviz transformers sentencepiece protobuf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from bertviz import head_view
import torch
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

device = torch.device("mps" if torch.mps.is_available() else "cpu")
model = model.to(device)

# CoT prompt
question = "Tom has 5 pencils, gives 2 to Jerry, and then buys 3 more. How many pencils does he have?"
prompt = f"Q: {question}\nA: Let's think step by step."

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate with attention
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        output_attentions=True,
        return_dict_in_generate=True
    )

# Collect tokenized output
tokens = tokenizer.convert_ids_to_tokens(outputs.sequences[0])

outputs = model(**inputs, output_attentions=True)

# Extract encoder-decoder attention from last layer
encoder_decoder_attn = outputs.cross_attentions[-1][0].detach().cpu()

# Visualize with BERTViz
head_view(encoder_decoder_attn, tokens[:len(inputs['input_ids'][0])], tokens)


Loading checkpoint shards: 100%|██████████| 2/2 [01:05<00:00, 32.72s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 20.25 GB, other allocations: 464.00 KB, max allowed: 20.40 GB). Tried to allocate 172.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).